In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# !pip install aiosmtpd
# !uv pip install aiosmtpd

In [1]:
# stdlib
import os
import re

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

'python'

In [2]:
# syft absolute
import syft as sy
from syft.service.notifier.notifier_enums import NOTIFIERS

# third party
from helpers import create_user
from helpers import make_user
from helpers import save_users
from helpers import load_users
from helpers import SENDER
from helpers import create_smtp_test_server
from helpers import emails_table

In [3]:
smtp_server = create_smtp_test_server()

In [4]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080
Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Will watch for changes in these directories: ['/home/ionesio/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [25210] using WatchFiles


Waiting for server to start...WARN: private key is based on server name: bigquery-high in dev_mode. Don't run this in production.
INFO:     127.0.0.1:41308 - "GET /api/v2/metadata HTTP/1.1" 200 OK
 Done.


INFO:     Started server process [25224]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


SyftInfo: You have launched a development server at http://0.0.0.0:8080.It is intended only for local use.

In [5]:
high_client = sy.login(
    url="http://localhost:8080", email="info@openmined.org", password="changethis"
)

INFO:     127.0.0.1:48736 - "GET /api/v2/metadata HTTP/1.1" 200 OK
INFO:     127.0.0.1:48736 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:48736 - "GET /api/v2/api?verify_key=1e5ec44a179d21e0a0721ea7a670b9875d3946c8604f4623903fcafafe03c8e7&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:48746 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [6]:
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="1025",
)
high_client.api.services.settings.disable_notifications()  # Turn off OnBoard Email Notifications

INFO:     127.0.0.1:48752 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:48766 - "POST /api/v2/api_call HTTP/1.1" 200 OK


In [7]:
num_users = 5
users = []
email_disable_index = 0
for i in range(num_users):
    user = make_user()
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    users.append(user)

INFO:     127.0.0.1:45856 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45864 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "GET /api/v2/api?verify_key=54b61830d9ef1c5999969e563f85746a67dc89cf4ca0484b3d50cdc8b428c7b8&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:45890 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <veronica-brown-fake@openmined.org>
INFO:     127.0.0.1:45906 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45920 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45924 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:45876 - "GET /api/v2/api?verify_key=5f2da9b1cdeacc8a0a

In [8]:
pattern = r"syft_client\.reset_password\(token='(.*?)', new_password=.*?\)"                                                                                                                                        

for user in users:
    user.client = high_client
    if user.email_disabled:
        high_client.api.services.settings.disable_notifications()
    else:
        high_client.api.services.settings.enable_notifications()
    
    user.client.forgot_password(email=user.email)

    if user.email_disabled:
        assert emails_table.get(user.email, None) is None
    else:
        assert emails_table.get(user.email, None) is not None
        recv_email = emails_table.get(user.email, None)
        assert recv_email.email_from == SENDER
        assert recv_email.email_to[0] == user.email

        token = re.search(pattern, recv_email.email_content).group(1)
        new_password = user.make_new_password()
        result = user.client.reset_password(token=token, new_password=new_password)
        assert isinstance(result, sy.SyftSuccess)

INFO:     127.0.0.1:35830 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:35830 - "GET /api/v2/api?verify_key=54b61830d9ef1c5999969e563f85746a67dc89cf4ca0484b3d50cdc8b428c7b8&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:42374 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <veronica-brown-fake@openmined.org>
INFO:     127.0.0.1:42388 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:35830 - "POST /api/v2/forgot_password HTTP/1.1" 200 OK
INFO:     127.0.0.1:35830 - "POST /api/v2/login HTTP/1.1" 200 OK
INFO:     127.0.0.1:35830 - "GET /api/v2/api?verify_key=5f2da9b1cdeacc8a0a867f770edb6ed50a4c0b4f80e3d8236e9c7be9a9e143be&communication_protocol=dev HTTP/1.1" 200 OK
INFO:     127.0.0.1:42392 - "POST /api/v2/api_call HTTP/1.1" 200 OK
Logged into <bigquery-high: High side Datasite> as <adam-rice-fake@openmined.org>
INFO:     127.0.0.1:42406 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:3583

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [25224]
INFO:     Started server process [26362]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [26362]
INFO:     Started server process [26987]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [ ]:
server.land()
smtp_server.stop()